<a href="https://colab.research.google.com/github/UnknwonD/project_ai/blob/main/project_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data 압축 풀기

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from glob import glob

In [ ]:
# %cd /content/drive/MyDrive/project_ai

# !unzip -qq "/content/drive/MyDrive/project_ai/VL_damage_part.zip"

In [ ]:
# %cd /content/drive/MyDrive/project_ai

# !unzip -qq "/content/drive/MyDrive/project_ai/VS_damage_part_jpg.zip"

/content/drive/MyDrive/project_ai


In [ ]:
filepaths1 = list(glob('/content/drive/MyDrive/project_ai/damage_part_json/*.json'))
print("json 파일 갯수 : {}".format(len(filepaths1)))

json 파일 갯수 : 17248


In [ ]:
filepaths2 = list(glob('/content/drive/MyDrive/project_ai/damage_part_jpg/*.jpg'))
print("jpg 파일 갯수 : {}".format(len(filepaths2)))

jpg 파일 갯수 : 17248


# 마스크 이미지 만들기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow
from PIL import Image

import pandas as pd
import numpy as npA
import json
import cv2
import os

In [ ]:
# 이미지 파일 리사이즈 하는 함수 (원본이미지파일 경로, mask이미지인가[폴더구분위함], 가로세로 크기)
def resize_jpg(url, mask=False, w=300):
  save_path = '/content/drive/MyDrive/project_ai/resized/'  # 저장할 이미지 경로

  # 저장할 경로 없으면 생성
  if not os.path.exists(save_path):
      os.mkdir(save_path)

  #원본 이미지 경로의 모든 이미지 list 지정
  data_list = os.listdir(url + "damage_part_jpg/") if not mask else os.listdir(url + "masked/")
  print(len(data_list))

  # 모든 이미지 resize 후 저장하기
  for token in data_list:
      # 이미지 열기
      im = Image.open(url + token)

      # 이미지 resize
      im = im.resize((w, w))

      # 이미지 JPG로 저장
      im = im.convert('RGB')
      im.save(save_path+token)
      print('end ::: ' + token)

# 파일 이름 리스트로 저장하는 함수
def extract_file_name(url):
  file_list = os.listdir(url)
  file_name = []
  for file in file_list:
    file_name.append(file.split('.')[0])
  return file_name

In [ ]:
files = extract_file_name("/content/drive/MyDrive/project_ai/damage_part_jpg/")

for file in files:
  imgfile = "/content/drive/MyDrive/project_ai/damage_part_jpg/{}.jpg".format(file)
  img = cv2.imread(imgfile)

  with open('/content/drive/MyDrive/project_ai/damage_part_json/{}.json'.format(file), 'r') as f:
    json_data = json.load(f)
  mask_num = 0
  for j in range(len(json_data["annotations"])):
    if json_data["annotations"][j]["damage"] == None:
      mask_num += 1

  img[img!=256] = 0
  save_path = '/content/drive/MyDrive/project_ai/masked/'

  if not os.path.exists(save_path):
    os.mkdir(save_path)

  for i in range(len(json_data["annotations"])-mask_num):
    area = np.array(json_data["annotations"][i]["segmentation"][0][0])
    filled = cv2.fillPoly(img, pts = [area], color = (255, 255, 255))
    
    cv2.imwrite("/content/drive/MyDrive/project_ai/masked/{}_mask.jpg".format(file), filled)


# Project


In [ ]:
## 라이브러리 불러오기
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## 데이터 불러오기
dir_data = '/content/drive/MyDrive/project3/dataset' 

path_dir_x = '/content/drive/MyDrive/project3/dataset/x/'
file_list_x = os.listdir(path_dir_x)
path_dir_y = '/content/drive/MyDrive/project3/dataset/y/'
file_list_y = os.listdir(path_dir_y)

for png in file_list_x:
    img_input = Image.open(path_dir_x + png)
for png in file_list_y:
    img_label = Image.open(path_dir_y + png)


# name_label = 'y/*.png'
# name_input = 'x/*.png'

# img_label = Image.open(os.path.join(dir_data, name_label))
# img_input = Image.open(os.path.join(dir_data, name_input))

ny, nx = (1508, 754)
nframe = 30

## train/test/val 폴더 생성
nframe_train = 24
nframe_val = 3
nframe_test = 3

dir_save_train = os.path.join(dir_data, 'train')
dir_save_val = os.path.join(dir_data, 'val')
dir_save_test = os.path.join(dir_data, 'test')

if not os.path.exists(dir_save_train):
    os.makedirs(dir_save_train)

if not os.path.exists(dir_save_val):
    os.makedirs(dir_save_val)

if not os.path.exists(dir_save_test):
    os.makedirs(dir_save_test)

## 전체 이미지 30개를 섞어줌
id_frame = np.arange(nframe)
np.random.shuffle(id_frame)

In [ ]:
## 선택된 train 이미지를 npy 파일로 저장
offset_nframe = 0

for i in range(nframe_train):
  png_x = file_list_x[id_frame[i + offset_nframe]]
  img_input = Image.open(path_dir_x + png_x)
  input_ = np.asarray(img_input)
  png_x = png_x.split('.')[0]
  np.save(os.path.join(dir_save_train, 'input_%03d.npy' % i), input_)
  
  png_y = file_list_y[id_frame[i + offset_nframe]]
  img_label = Image.open(path_dir_y + png_y)
  label_ = np.asarray(img_label)
  png_y = png_y.split('.')[0]
  np.save(os.path.join(dir_save_train, 'label_%03d.npy' % i), label_)

In [ ]:
## 선택된 val 이미지를 npy 파일로 저장
offset_nframe = nframe_train

for i in range(nframe_val):
  png_x = file_list_x[id_frame[i + offset_nframe]]
  img_input = Image.open(path_dir_x + png_x)
  input_ = np.asarray(img_input)
  png_x = png_x.split('.')[0]
  np.save(os.path.join(dir_save_val, 'input_%03d.npy' % i), input_)
  
  png_y = file_list_y[id_frame[i + offset_nframe]]
  img_label = Image.open(path_dir_y + png_y)
  label_ = np.asarray(img_label)
  png_y = png_y.split('.')[0]
  np.save(os.path.join(dir_save_val, 'label_%03d.npy' % i), label_)

In [ ]:
## 선택된 val 이미지를 npy 파일로 저장
offset_nframe = nframe_train

for i in range(nframe_val):
  png_x = file_list_x[id_frame[i + offset_nframe]]
  img_input = Image.open(path_dir_x + png_x)
  input_ = np.asarray(img_input)
  png_x = png_x.split('.')[0]
  np.save(os.path.join(dir_save_val, 'input_%03d.npy' % i), input_)
  
  png_y = file_list_y[id_frame[i + offset_nframe]]
  img_label = Image.open(path_dir_y + png_y)
  label_ = np.asarray(img_label)
  png_y = png_y.split('.')[0]
  np.save(os.path.join(dir_save_val, 'label_%03d.npy' % i), label_)

In [ ]:
## 이미지 시각화

plt.subplot(122)
plt.imshow(label_, cmap='gray')
plt.title('label')

plt.subplot(121)
plt.imshow(input_, cmap='gray')
plt.title('input')

plt.show()

In [ ]:
## 한 이미지의 분포

plt.subplot(122)
plt.hist(label_.flatten(), bins=20)
plt.title('label')

plt.subplot(121)
plt.hist(input_.flatten(), bins=20)
plt.title('input')

plt.tight_layout()
plt.show()

In [ ]:
## 라이브러리 불러오기
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt

from torchvision import transforms, datasets

## 네트워크 구축하기
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Convolution + BatchNormalization + Relu 정의하기
        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True): 
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        # 수축 경로(Contracting path)
        self.enc1_1 = CBR2d(in_channels=3, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # 확장 경로(Expansive path)
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=1, stride=1, padding=0, bias=True)
    
    # forward 함수 정의하기
    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)

        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x

In [ ]:
label_.shape

In [ ]:
input_.shape

In [ ]:
# 데이터로더 잘 구현되었는지 확인

dataset_train = Dataset(data_dir=dir_save_train)
data = dataset_train.__getitem__(0) # 한 이미지 불러오기
input = data['input']
label = data['label']

In [ ]:
print(input.shape)
print(input.ndim)

In [ ]:
print(label.shape)
print(label.ndim)

In [ ]:
# 불러온 이미지 시각화

plt.subplot(122)
plt.imshow(label.reshape(754, 1508, 3), cmap='gray')
plt.title('label')

plt.subplot(121)
plt.imshow(input.reshape(754, 1508, 3), cmap='gray')
plt.title('input')

plt.show()

In [ ]:
# 불러온 이미지 시각화

plt.subplot(122)
plt.imshow(label.reshape(754, 1508, 3), cmap='gray')
plt.title('label')

plt.subplot(121)
plt.imshow(input.reshape(754, 1508, 3), cmap='gray')
plt.title('input')

plt.show()

In [ ]:
# 트랜스폼 잘 구현되었는지 확인
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])
dataset_train = Dataset(data_dir=dir_save_train, transform=transform)
data = dataset_train.__getitem__(0) # 한 이미지 불러오기
input = data['input']
label = data['label']


# 불러온 이미지 시각화
plt.subplot(122)
plt.hist(label.flatten(), bins=20)
plt.title('label')

plt.subplot(121)
plt.hist(input.flatten(), bins=20)
plt.title('input')

plt.tight_layout()
plt.show()

In [ ]:
# 트랜스폼 잘 구현되었는지 확인
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])
dataset_train = Dataset(data_dir=dir_save_train, transform=transform)
data = dataset_train.__getitem__(0) # 한 이미지 불러오기
input = data['input']
label = data['label']


# 불러온 이미지 시각화
plt.subplot(122)
plt.hist(label.flatten(), bins=20)
plt.title('label')

plt.subplot(121)
plt.hist(input.flatten(), bins=20)
plt.title('input')

plt.tight_layout()
plt.show()

In [ ]:
# 훈련 파라미터 설정하기
lr = 1e-3
batch_size = 4
num_epoch = 20

base_dir = './drive/MyDrive/DACrew/unet'
data_dir = dir_data
ckpt_dir = os.path.join(base_dir, "checkpoint")
log_dir = os.path.join(base_dir, "log")


# 훈련을 위한 Transform과 DataLoader
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])

dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=transform)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=8)

dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=8)

# 네트워크 생성하기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet().to(device)

# 손실함수 정의하기
fn_loss = nn.BCEWithLogitsLoss().to(device)

# Optimizer 설정하기
optim = torch.optim.Adam(net.parameters(), lr=lr)

# 그밖에 부수적인 variables 설정하기
num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

num_batch_train = np.ceil(num_data_train / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

# 그 밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

# Tensorboard 를 사용하기 위한 SummaryWriter 설정
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

# 네트워크 학습시키기
st_epoch = 0
# 학습한 모델이 있을 경우 모델 로드하기
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim) 

for epoch in range(st_epoch + 1, num_epoch + 1):
        net.train()
        loss_arr = []

        for batch, data in enumerate(loader_train, 1):
            # forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)

            output = net(input)

            # backward pass
            optim.zero_grad()

            loss = fn_loss(output, label)
            loss.backward()

            optim.step()

            # 손실함수 계산
            loss_arr += [loss.item()]

            print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_class(output))

            writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

        writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

        with torch.no_grad():
            net.eval()
            loss_arr = []

            for batch, data in enumerate(loader_val, 1):
                # forward pass
                label = data['label'].to(device)
                input = data['input'].to(device)

                output = net(input)

                # 손실함수 계산하기
                loss = fn_loss(output, label)

                loss_arr += [loss.item()]

                print("VALID: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                      (epoch, num_epoch, batch, num_batch_val, np.mean(loss_arr)))

                # Tensorboard 저장하기
                label = fn_tonumpy(label)
                input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
                output = fn_tonumpy(fn_class(output))

                writer_val.add_image('label', label, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
                writer_val.add_image('input', input, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
                writer_val.add_image('output', output, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')

        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

        # epoch 50마다 모델 저장하기
        if epoch % 50 == 0:
            save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)

        writer_train.close()
        writer_val.close()